# Поиск собственных чисел матрицы

## Степенной метод

In [1]:
import numpy as np
from tqdm import trange

import lesson_2

In [2]:
def generate_matrix(eigvals):
    """
    Генерирует случайную матрицу с заданными собственными числами
    """
    J = np.diag(eigvals)

    S = np.random.random((len(eigvals), len(eigvals)))

    return np.linalg.inv(S) @ J @ S

In [3]:
def calc_max_abs_eigval(A, r0=None, iters=10000):
    """
    Степенной метод нахождения максимального собственного значения

    """
    if r0 is None:
        r0 = np.random.random((A.shape[0],))

    r = r0.copy()

    for _ in trange(iters):
        r = A @ r
        r = r / lesson_2.get_vector_norm_3(r)

    return np.dot(r, A @ r) / np.dot(r, r)

In [6]:
# Заданные собственные числа
l = np.array([1, 2, 4, 8, 16, 32, 64])
A = generate_matrix(l)

print(l)
A

[ 1  2  4  8 16 32 64]


array([[  27.512143  ,    2.06879751,   19.75701698,   22.013218  ,
           3.03583144,   11.88087641,    9.62878623],
       [   5.76628178,    5.67511699,   -0.57675576,    6.15750968,
           8.57636379,   -0.72084941,    3.66868586],
       [  98.37860281,  137.09394734,   59.11673171,   62.3713496 ,
         102.71552977,   29.85476199,   86.89592785],
       [ -68.51688766, -118.49202257,  -38.24754197,  -41.58040712,
         -88.72206039,  -34.33088503,  -76.92689444],
       [ 124.60425032,  196.11093944,   61.4560529 ,   77.78931639,
         156.11498348,   42.58060842,  124.4779362 ],
       [  66.00806555,   73.25535919,   39.00676602,   53.77120943,
          63.44685663,   44.01932427,   60.97064928],
       [-163.44073837, -171.68212964,  -94.38038477, -119.11976216,
        -143.3551864 ,  -57.25551344, -123.85789233]])

In [7]:
np.abs(np.linalg.eigvals(A)).max(), calc_max_abs_eigval(A)

100%|██████████| 10000/10000 [00:00<00:00, 129876.73it/s]


(64.00000000000011, 63.99999999999996)

In [23]:
# Случайная матрица с действительными собственными числами
A = generate_matrix(np.random.random(10))

np.abs(np.linalg.eigvals(A)).max(), calc_max_abs_eigval(A)

100%|██████████| 10000/10000 [00:00<00:00, 136990.41it/s]


(0.7333445240292409, 0.7333445240292401)

In [9]:
# Работа с комплексными числами
A = np.random.random((10, 10))

print(np.linalg.eigvals(A))

np.abs(np.linalg.eigvals(A)).max(), calc_max_abs_eigval(A)

[ 4.69473223+0.j         -0.93417033+0.j          0.92001381+0.j
 -0.41413735+0.j          0.27423755+0.05913139j  0.27423755-0.05913139j
 -0.04298094+0.65181669j -0.04298094-0.65181669j -0.10583629+0.54314632j
 -0.10583629-0.54314632j]


100%|██████████| 10000/10000 [00:00<00:00, 126566.24it/s]


(4.694732230777673, 4.694732230777667)

Видно, что поиск максимального по модулю собственного числа степенным методом выдаёт тот же результат, что и честный расчёт.

## Метод вращений для поиска спектра самосопряженной матрицы

In [4]:
def findMax(a):
    m = [a[0][1], 0, 1]
    for i in range(len(a)):
        for j in range(len(a)):
            if j > i:
                if a[i][j] > m[0]:
                    m = [a[i][j], i, j]
    return m

In [5]:
def null(Q, t=None):
    if t == None:
        q = Q.tolist()
    for i in range(len(q)):
        for j in range(len(q[i])):
            if abs(q[i][j]) < 1.0e-7:
                q[i][j] = 0.
    return q

In [10]:
def complete(T):
    q = T["A"][-1]
    y = True
    for i in range(len(q)):
        for j in range(len(q[i])):
            if i != j:
                y = y and abs(q[i][j]) < 1.0e-7
    return y

In [116]:
from math import sin, cos, atan
from numpy import matrix


def spectr(A):
    """
    Метод вращений для симметричной матрицы

    Возвращает кортеж:
        - Первый элемент - список собственных значений
        - Второй - матрица, строки которой являются соответствующие собственные векторы
    """

    E = np.eye(*A.shape)
    T = {"H": [], "A": [A.tolist()]}

    i = 0
    while not complete(T) and i <= 9:
        i += 1

        A = T["A"][i - 1]

        p = findMax(A)
        k, m = p[1], p[2]

        f = 1. / 2 * atan(2. * A[k][m] / (A[k][k] - A[m][m]))

        H = E.copy().tolist()
        H[k][k] = cos(f)
        H[k][m] = -sin(f)
        H[m][k] = sin(f)
        H[m][m] = cos(f)
        T["H"].append(H)

        H = matrix(H)
        A = matrix(A)

        Ai = null((H.T * A) * H)
        T["A"].append(Ai)

    V = matrix(T["H"][0])
    for h in T["H"][1:]:
        V = V * h

    return np.diag(A), V.T / np.max(V, axis=0)

In [121]:
# l = np.random.random(10)
np.sort(l)

array([0.13164291, 0.30096253, 0.37416821, 0.46817039, 0.48499203,
       0.50596918, 0.60599736, 0.77090845, 0.88227796, 0.94991114])

In [122]:
V = spectr(A)

L1 = 0.56779088
L2 = 0.26676685
L3 = 0.64716179
L4 = 0.57685150
L5 = 1.56597615
L6 = 0.36818354
L7 = 0.44939555
L8 = 0.11450090
L9 = 0.26449336
L10 = 0.65387962
V1 :
    1.00000000
    0.00000000
    -0.21475266
    0.00000000
    -1.19761201
    0.00000000
    -0.17526766
    -0.63558088
    0.00000000
    -0.17116552
V1 :
    0.24859465
    1.00000000
    0.00000000
    0.00000000
    0.22893274
    0.00000000
    0.00000000
    0.10124033
    0.00000000
    -0.52536492
V1 :
    -0.28704188
    0.00000000
    1.00000000
    0.00000000
    -0.20170169
    0.00000000
    -0.28009451
    -0.25853869
    0.00000000
    -0.27353889
V1 :
    0.00000000
    0.00000000
    0.00000000
    1.00000000
    0.00000000
    0.00000000
    0.00000000
    0.00000000
    0.00000000
    0.00000000
V1 :
    1.00000000
    0.00000000
    0.83674805
    0.00000000
    0.21914712
    0.36220930
    0.68290131
    0.50978410
    0.29632447
    0.66691798
V1 :
    -0.12407885
    0.00000000
    -0.10382274
 

In [123]:
V / np.max(V, axis=0)

matrix([[ 1.        ,  0.24859465, -0.28704188,  0.        ,  1.        ,
         -0.12407885, -0.33394212, -0.16234322, -0.0971434 , -0.47318472],
        [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.76226521],
        [-0.21475266,  0.        ,  1.        ,  0.        ,  0.83674805,
         -0.10382274,  0.        ,  0.        , -0.08128455,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [-1.19761201,  0.22893274, -0.20170169,  0.        ,  0.21914712,
         -0.02719152, -0.30752988, -0.62096812, -0.0212887 , -0.43575946],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.3622093 ,
          1.        ,  0.        ,  0.        , -0.03518624,  0.        ],
        [-0.17526766,  0.        , -0.28009451,  0.        ,  0.68290131,
         -0.08473361,  1.       

In [124]:
np.eig()

AttributeError: module 'numpy' has no attribute 'eig'

In [125]:
#A = generate_matrix(l)

eigvals, eigs = spectr(A)
eigvals, eigs

L1 = 0.56779088
L2 = 0.26676685
L3 = 0.64716179
L4 = 0.57685150
L5 = 1.56597615
L6 = 0.36818354
L7 = 0.44939555
L8 = 0.11450090
L9 = 0.26449336
L10 = 0.65387962
V1 :
    1.00000000
    0.00000000
    -0.21475266
    0.00000000
    -1.19761201
    0.00000000
    -0.17526766
    -0.63558088
    0.00000000
    -0.17116552
V1 :
    0.24859465
    1.00000000
    0.00000000
    0.00000000
    0.22893274
    0.00000000
    0.00000000
    0.10124033
    0.00000000
    -0.52536492
V1 :
    -0.28704188
    0.00000000
    1.00000000
    0.00000000
    -0.20170169
    0.00000000
    -0.28009451
    -0.25853869
    0.00000000
    -0.27353889
V1 :
    0.00000000
    0.00000000
    0.00000000
    1.00000000
    0.00000000
    0.00000000
    0.00000000
    0.00000000
    0.00000000
    0.00000000
V1 :
    1.00000000
    0.00000000
    0.83674805
    0.00000000
    0.21914712
    0.36220930
    0.68290131
    0.50978410
    0.29632447
    0.66691798
V1 :
    -0.12407885
    0.00000000
    -0.10382274
 

ValueError: too many values to unpack (expected 2)

In [126]:
np.sort(eigvals), np.sort(np.linalg.eigvals(A))

(array([-0.60258237,  0.24870163,  0.27549632,  0.37439262,  0.40429395,
         0.41557708,  0.70186818,  0.95860009,  1.2866183 ,  1.41242933]),
 array([0.13164291, 0.30096253, 0.37416821, 0.46817039, 0.48499203,
        0.50596918, 0.60599736, 0.77090845, 0.88227796, 0.94991114]))